<div style='background-color:orange'>
<a id="TableOfContents"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            TABLE OF CONTENTS:
        </i></b></h1>
    <li><a href='#imports'>Imports</a>
    <li><a href="#acquire">Acquire</a>
    <li><a href='#prepare'>Prepare</a>
    <li><a href="#wrangle">Wrangle</a>
    <li><a href='#misc'>Miscellaneous</a>
    </li>
</div>

<div style='background-color:orange'>
<a id="imports"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Imports
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# Regex
import re

# .py files
import wrangle as w

<div style='background-color:orange'>
<a id="acquire"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Acquire
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

Acquire all crashes with motorcycles in Texas from the 2018 - 2022 via <a href='https://cris.dot.state.tx.us/public/Query/app/query-builder'>CRIS Query</a> data pull

- Master Vanilla Shape:
    - Rows: 81,153
    - Columns: 233

In [2]:
# Read the .csv exported file from CRIS Query
master = pd.read_csv('master_list.csv', index_col=0)
master.shape

/var/folders/6x/kctcvd8x6l18t57t92kmqy6w0000gn/T/ipykernel_89219/1769861746.py:2: DtypeWarning: Columns (128) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv('master_list.csv', index_col=0)


(81153, 233)

<div style='background-color:orange'>
<a id="prepare"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Prepare
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    <li><a href='#preparenormalize'>Normalize Columns</a>
    <li><a href='#preparenulls'>Null Handling</a>
    <li><a href='#preparesvc'>Identify Single Vehicle Crashes(SVC)</a>
    <li><a href='#prepareremovespecific'>Remove specific Columns</a>
    </li>
</div>

<a id='preparenormalize'></a>
<h3><b><i>
    Normalize Columns:
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [3]:
# Reset the index to ensure crash id's are not the index
master.reset_index(inplace=True)
master.head(2)

,Crash ID,$1000 Damage to Any One Person's Property,Active School Zone Flag,Adjusted Average Daily Traffic Amount,Adjusted Percentage of Average Daily Traffic For Trucks,Adjusted Roadway Part,Agency,At Intersection Flag,Average Daily Traffic Amount,Average Daily Traffic Year,...,Person Non-Suspected Serious Injury Count,Person Not Injured Count,Person Possible Injury Count,Person Restraint Used,Person Suspected Serious Injury Count,Person Time of Death,Person Total Injury Count,Person Type,Person Unknown Injury Count,Physical Location of An Occupant
0,16189632,No,NO,No Data,No Data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,No Data,No Data,...,0,0,0,97 - NOT APPLICABLE,1,No Data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
1,16188701,Yes,NO,No Data,No Data,1 - MAIN/PROPER LANE,ABILENE POLICE DEPARTMENT,True,No Data,No Data,...,0,1,0,1 - SHOULDER & LAP BELT,0,No Data,0,1 - DRIVER,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER


In [4]:
# Replace all whitespace and lowercase everything
master.columns = master.columns.str.replace(' ', '_').str.lower()
master.columns

Index(['crash_id', '$1000_damage_to_any_one_person's_property',
       'active_school_zone_flag', 'adjusted_average_daily_traffic_amount',
       'adjusted_percentage_of_average_daily_traffic_for_trucks',
       'adjusted_roadway_part', 'agency', 'at_intersection_flag',
       'average_daily_traffic_amount', 'average_daily_traffic_year',
       ...
       'person_non-suspected_serious_injury_count', 'person_not_injured_count',
       'person_possible_injury_count', 'person_restraint_used',
       'person_suspected_serious_injury_count', 'person_time_of_death',
       'person_total_injury_count', 'person_type',
       'person_unknown_injury_count', 'physical_location_of_an_occupant'],
      dtype='object', length=234)

---

<a id='preparenulls'></a>
<h3><b><i>
    Null Handling:
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [5]:
# Identify columns with null values first
columns_with_nulls = master.columns[master.isna().sum() > 0]
master[columns_with_nulls].isna().sum()

case_id                             9939
intersecting_street_name           47292
street_name                            2
carrier's_primary_address_-_zip    80077
driver_license_endorsements          378
driver_license_restrictions          378
vehicle_towed_by                    1692
vehicle_towed_to                    2168
charge                                38
citation                              55
dtype: int64

In [6]:
# Convert all nulls to 'No Data' to maintain consistency with
# null value formatting from original data pull
master.fillna('No Data', inplace=True)

In [7]:
# Ensure all occurances of 'No Data' is consistent as 'no data' 
# and isn't varients like 'NO DATA', 'No DaTa', etc.
master = master.replace(to_replace=re.compile(r'.*no\s*data.*', re.IGNORECASE), value='no data', regex=True)
master.head(2)

,crash_id,$1000_damage_to_any_one_person's_property,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,adjusted_roadway_part,agency,at_intersection_flag,average_daily_traffic_amount,average_daily_traffic_year,...,person_non-suspected_serious_injury_count,person_not_injured_count,person_possible_injury_count,person_restraint_used,person_suspected_serious_injury_count,person_time_of_death,person_total_injury_count,person_type,person_unknown_injury_count,physical_location_of_an_occupant
0,16189632,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,1,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
1,16188701,Yes,NO,no data,no data,1 - MAIN/PROPER LANE,ABILENE POLICE DEPARTMENT,True,no data,no data,...,0,1,0,1 - SHOULDER & LAP BELT,0,no data,0,1 - DRIVER,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER


---

<a id='preparesvc'></a>
<h3><b><i>
    Identify Single Vehicle Crashes(SVC):
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [8]:
# Using the function from wrangle.py,
# Get only single motorcycle crashes
svcs = w.get_single_motorcycle_crashes(master)
svcs.shape

(14548, 234)

In [9]:
# Ensure each crash is unique (Meaning each crash only has one vehicle)
svcs.crash_id.nunique()

14548

In [10]:
# Ensure only motorcyclists are identified
svcs.person_type.value_counts()

person_type
5 - DRIVER OF MOTORCYCLE TYPE VEHICLE    14548
Name: count, dtype: int64

In [46]:
svcs.vehicle_body_style.value_counts()

vehicle_body_style
MC - MOTORCYCLE           14517
PM - POLICE MOTORCYCLE       31
Name: count, dtype: int64

- Single Vehicle Crashes (SVCs // Motorcyclists) Shape:
    - Rows: 14,548
    - Columns: 234

---

<a id='prepareremovespecific'></a>
<h3><b><i>
    Remove Specific Columns:
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [115]:
svcs = pd.read_csv('svcs.csv', index_col=0)
svcs.head(3)

,crash_id,$1000_damage_to_any_one_person's_property,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,adjusted_roadway_part,agency,at_intersection_flag,average_daily_traffic_amount,average_daily_traffic_year,...,person_non-suspected_serious_injury_count,person_not_injured_count,person_possible_injury_count,person_restraint_used,person_suspected_serious_injury_count,person_time_of_death,person_total_injury_count,person_type,person_unknown_injury_count,physical_location_of_an_occupant
0,16189632,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,1,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
11,16203470,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,1,97 - NOT APPLICABLE,0,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
14,16191458,Yes,NO,no data,no data,1 - MAIN/PROPER LANE,WILLIAMSON COUNTY SHERIFF'S OFFICE,False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,0,no data,0,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,1,1 - FRONT LEFT OR MOTORCYCLE DRIVER


In [116]:
cols_to_remove = [
    'driver_drug_specimen_type',
    'person_drug_specimen_type',
    'person_drug_test_result',
    'driver_alcohol_result',
    'driver_alcohol_specimen_type',
    'person_alcohol_result',
    'person_alcohol_specimen_type_taken',
    'person_blood_alcohol_content_test_result',
    'crash_non-suspected_serious_injury_count',
    'crash_not_injured_count',
    'crash_possible_injury_count',
    'crash_suspected_serious_injury_count',
    'crash_total_injury_count',
    'crash_unknown_injury_count',
    'unit_non-suspected_serious_injury_count',
    'unit_not_injured_count',
    'unit_possible_injury_count',
    'unit_suspected_serious_injury_count',
    'unit_total_injury_count',
    'unit_unknown_injury_count',
    'person_non-suspected_serious_injury_count',
    'person_not_injured_count',
    'person_possible_injury_count',
    'person_suspected_serious_injury_count',
    'person_total_injury_count',
    'person_unknown_injury_count',
    'crash_death_count',
    'driver_time_of_death',
    'unit_death_count',
    'person_death_count',
    'person_time_of_death',
    'autonomous_level_engaged',
    'autonomous_unit_-_reported',
    'school_bus_flag',
    'bus_type',
    'cmv_actual_gross_weight',
    'cmv_cargo_body_type',
    'cmv_carrier_id_type',
    'cmv_disabling_damage_-_power_unit',
    'cmv_gvwr',
    'cmv_hazmat_release_flag',
    'cmv_intermodal_shipping_container_permit',
    'cmv_rgvw',
    'cmv_roadway_access',
    'cmv_sequence_of_events_1',
    'cmv_sequence_of_events_2',
    'cmv_sequence_of_events_3',
    'cmv_sequence_of_events_4',
    'cmv_total_number_of_axles',
    'cmv_total_number_of_tires',
    'cmv_trailer_disabling_damage',
    'cmv_trailer_gvwr',
    'cmv_trailer_rgvw',
    'cmv_trailer_type',
    'cmv_vehicle_operation',
    'cmv_vehicle_type',
    'vehicle_cmv_flag',
    'first_harmful_event',
    'first_harmful_event_involvement',
    'hazmat_class_1_id',
    'hazmat_class_2_id',
    'hazmat_id_number_1_id',
    'hazmat_id_number_2_id',
    'responder_struck_flag',
    'unit_description',
    'person_airbag_deployed',
    'person_ejected',
    'person_restraint_used',
    'highway_lane_design_for_hov,_railroads,_and_toll_roads',
    'railroad_company',
    'railroad_flag',
    'bridge_detail',
    'feature_crossed_by_bridge',
    'on_bridge_service_type',
    'under_bridge_service_type',
    'construction_zone_flag',
    'construction_zone_workers_present_flag',
    'commercial_motor_vehicle_flag',
    'date_arrived',
    'date_notified',
    'date_roadway_cleared',
    'date_scene_cleared',
    'direction_of_traffic'
]

In [117]:
len(cols_to_remove)

83

In [119]:
svcs.drop(columns=cols_to_remove, inplace=True)

In [120]:
svcs, new_dict = w.drop_nullpct_alternate(svcs, 0.987)

In [121]:
drop_negone_pct_dict = {
    'column_name' : [],
    'percent_nodata' : []
}
for col in svcs:
    pct = (svcs[col] == -1).sum() / svcs.shape[0]
    if pct > 0.987:
        svcs = svcs.drop(columns=col)
        drop_negone_pct_dict['column_name'].append(col)
        drop_negone_pct_dict['percent_nodata'].append(pct)

In [123]:
pd.DataFrame(drop_negone_pct_dict)

,column_name,percent_nodata
0,intersecting_road_speed_limit,1.0
1,secondary_crash_flag,1.0


In [125]:
svcs.to_csv('new_svcs.csv')

In [126]:
svcs.shape

(14548, 131)

<div style='background-color:orange'>
<a id="wrangle"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Wrangle
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [11]:
# Check functionality from wrangle.py
function_svcs = w.wrangle()
print(f'\033[35mWrangle Function:\033[0m {function_svcs.shape}\n\033[35mManual Preparation:\033[0m {svcs.shape}')

Wrangle Function: (14548, 234)
Manual Preparation: (14548, 234)


In [32]:
# Read all separated versions of files
all_person = pd.read_csv('2018_all_person.csv', index_col=0)
contributing_factors = pd.read_csv('2018_contributing_factors.csv', index_col=0)
crash_conditions = pd.read_csv('2018_crash_conditions.csv', index_col=0)
lat_long = pd.read_csv('2018_lat_long.csv', index_col=0)
road_stuff = pd.read_csv('2018_road_stuff.csv', index_col=0)
vehicle_info = pd.read_csv('2018_vehicle_info.csv', index_col=0)

In [33]:
def dataset_consistency(df):
    df.reset_index(inplace=True)
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    col_list = df.columns.to_list()
    target_variable_check = 'person_injury_severity' in col_list
    if target_variable_check == True:
        return col_list
    else:
        col_list.append('person_injury_severity')
        return col_list

In [34]:
# Create lists of column names WITH target variable column
all_person_col_list = dataset_consistency(all_person)
contributing_factors_col_list = dataset_consistency(contributing_factors)
crash_conditions_col_list = dataset_consistency(crash_conditions)
lat_long_col_list = dataset_consistency(lat_long)
road_stuff_col_list = dataset_consistency(road_stuff)
vehicle_info_col_list = dataset_consistency(vehicle_info)

In [ ]:
# Append additional columns for exploration


In [43]:
# Apply cols as a mask onto svcs dataframe
all_person_new = svcs[all_person_col_list]
contributing_factors_new = svcs[contributing_factors_col_list]
crash_conditions_new = svcs[crash_conditions_col_list]
lat_long_new = svcs[lat_long_col_list]
road_stuff_new = svcs[road_stuff_col_list]
vehicle_info_new = svcs[vehicle_info_col_list]

In [45]:
# Create .csvs for each of them for potential later use
#all_person_new.to_csv('all_person_master.csv')
#contributing_factors_new.to_csv('contributing_factors_master.csv')
#crash_conditions_new.to_csv('crash_conditions_master.csv')
#lat_long_new.to_csv('lat_long_master.csv')
#road_stuff_new.to_csv('road_stuff_master.csv')
#vehicle_info_new.to_csv('vehicle_info_master.csv')

<div style='background-color:orange'>
<a id="misc"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Miscellaneous
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [79]:
test = svcs
test, dropped_cols = w.drop_nullpct_alternate(test, 0.987)

In [80]:
test

,crash_id,$1000_damage_to_any_one_person's_property,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,adjusted_roadway_part,agency,at_intersection_flag,average_daily_traffic_amount,average_daily_traffic_year,...,person_non-suspected_serious_injury_count,person_not_injured_count,person_possible_injury_count,person_restraint_used,person_suspected_serious_injury_count,person_time_of_death,person_total_injury_count,person_type,person_unknown_injury_count,physical_location_of_an_occupant
0,16189632,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,1,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
11,16203470,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,1,97 - NOT APPLICABLE,0,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
14,16191458,Yes,NO,no data,no data,1 - MAIN/PROPER LANE,WILLIAMSON COUNTY SHERIFF'S OFFICE,False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,0,no data,0,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,1,1 - FRONT LEFT OR MOTORCYCLE DRIVER
18,16192023,Yes,NO,no data,no data,1 - MAIN/PROPER LANE,HARRIS COUNTY SHERIFF'S OFFICE,False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,1,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
22,16196720,No,NO,no data,no data,1 - MAIN/PROPER LANE,MCALLEN POLICE DEPARTMENT,False,no data,no data,...,1,0,0,97 - NOT APPLICABLE,0,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81125,19321499,Yes,NO,1211,6.9,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,1211,2019,...,1,0,0,97 - NOT APPLICABLE,0,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
81126,19323296,Yes,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,1,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
81140,19327850,No,NO,no data,no data,1 - MAIN/PROPER LANE,HARRIS COUNTY CONSTABLE PRECINCT 4,False,no data,no data,...,0,0,0,97 - NOT APPLICABLE,1,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER
81144,19330330,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,0,0,1,97 - NOT APPLICABLE,0,no data,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,0,1 - FRONT LEFT OR MOTORCYCLE DRIVER


In [82]:
percentagethingy = pd.DataFrame(dropped_cols)
percentagethingy[percentagethingy.percent_nodata != 1]

,column_name,percent_nodata
0,crossing_number,0.999313
43,possible_vehicle_defect_2,0.999656
46,vehicle_defect_2,0.999519


In [52]:
test.crash_time

0        1123
11       1316
14       2207
18       2045
22        307
         ... 
81125    1126
81126    2229
81140    1555
81144    1449
81152    1226
Name: crash_time, Length: 14548, dtype: int64

In [107]:
drop_nodata_pct_dict = {
    'column_name' : [],
    'percent_nodata' : []
}
for col in svcs:
    pct = (svcs[col] == -1).sum() / svcs.shape[0]
    if pct > 0.987:
        svcs = svcs.drop(columns=col)
        drop_nodata_pct_dict['column_name'].append(col)
        drop_nodata_pct_dict['percent_nodata'].append(pct)

In [109]:
drop_nodata_pct_dict = {
    'column_name' : [],
    'percent_nodata' : []
}
for col in svcs:
    pct = (svcs[col] == -1).sum() / svcs.shape[0]
    if pct > 0.987:
        svcs = svcs.drop(columns=col)
        drop_nodata_pct_dict['column_name'].append(col)
        drop_nodata_pct_dict['percent_nodata'].append(pct)

In [110]:
pd.DataFrame(drop_nodata_pct_dict)

,column_name,percent_nodata
